# Setup

## Imports

In [ ]:
from tqdm import tqdm_notebook

Custom Modules

In [ ]:
import vaidata, vaikeras

from vaiutils import path_consts, plot_images
from vaikaggle import dataset_consts
from vaivision import dream
from vaikeras import get_predictions
from vaikaggle import plot_prediction_density, plot_confusion_matrix
from vaimodels import VGG16

Keras Modules

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.backend import set_learning_phase, clear_session
from keras.layers import Conv2D, Dense, BatchNormalization, Dropout, Flatten

import keras.backend as K

## Define useful variables and functions

In [ ]:
for k, v in path_consts('StateFarm'):
    exec(k + '=v')
for k, v in dataset_consts(DIR_DATA, use_sample=False):
    exec(k + '=v')

In [ ]:
def pickle_load(filename, default=None, path=DIR_CHECKPOINTS):
    return vaidata.pickle_load(path + '/' + filename + '.p', default)

In [ ]:
def pickle_dump(filename, obj, path=DIR_CHECKPOINTS):
    vaidata.pickle_dump(path + '/' + filename + '.p', obj)

In [ ]:
def inflate_model(name=None, model=None):
        return vaikeras.inflate_model(DIR_CHECKPOINTS + '/' + name + '.h5', name,
                                      DIR_CHECKPOINTS + '/' + name + '-history.p', model)

In [ ]:
def get_images(filenames):
    return vaidata.get_images(filenames, (224, 224, 3), DIR_VALID)

## Get Data

In [ ]:
valid_batches = vaikeras.get_batches(DIR_VALID, ImageDataGenerator(), batch_size=1, shuffle=False)

## Get Model

In [ ]:
K.clear_session()
K.set_learning_phase(False)
model = inflate_model('FullyConvNetBW')
model.summary()

# Dream

## Dream all the convolutional layers

In [ ]:
for j, layer in tqdm_notebook(enumerate(model.layers)):
    if type(layer) is Conv2D:
        plot_images([dream(model, filter_idx=(j, i))
                     for i in tqdm_notebook(randint(0, layer.output_shape[-1], 16))])
        plot_images([dream(model, filter_idx=(j, -1), strength=j)])

## Dream all the Dense Layers

In [ ]:
for j, layer in tqdm_notebook(enumerate(model.layers)):
    if type(layer) == Dense:
        print(layer.name)
        plot_images([dream(model, filter_idx=(j, -1), strength=j)])

## Dream the final layer (categories)

In [ ]:
plot_images([dream(model, filter_idx=(-1, i), strength=100).mean(-1)
             for i in tqdm_notebook(range(10))], cmap='seismic')

# Single Image

## Follow an Image

In [ ]:
intermediate_model = Model(
    model.input, [layer.output for layer in model.layers if type(layer) == Conv2D])

In [ ]:
sample_image = valid_batches.next()[0][randint(0, valid_batches.batch_size)]
sample_activations = intermediate_model.predict(
    np.expand_dims(sample_image, 0))

In [ ]:
plot_images([sample_image])

In [ ]:
for s in sample_activations:
    plot_images([s[0, :, :, i] for i in randint(0, s.shape[-1], 16)], pixel_range=(s.min(), s.max()))

In [ ]:
plot_images([dream(model, sample_image, filter_idx=(-1, i), strength=10) for i in tqdm_notebook(range(10))])

## Saliency Maps 

In [ ]:
sample_image = valid_batches.next()[0][randint(0, valid_batches.batch_size)]

In [ ]:
input_img = model.input
grads = K.gradients(K.max(model.output, -1), input_img)[0]
saliency_map = K.function([input_img], [grads])

sample_saliency = saliency_map([np.expand_dims(sample_image, 0)])[0][0]

In [ ]:
plot_images([sample_image], flags='retain')
plot_images([np.linalg.norm(sample_saliency, axis=-1)], pixel_range=(sample_saliency.min()*0.1, sample_saliency.max()), cmap='seismic', flags='retain')
plt.axes().images[1].set_alpha(0.4)
plt.figure

# Predictions

In [ ]:
val_predictions = model.predict_generator(valid_batches, valid_batches.num_batches, verbose=1)

In [ ]:
val_losses = -np.log(val_predictions[range(valid_batches.samples), valid_batches.classes])

In [ ]:
def plot_predictions(mode):
    plot_images(get_images(get_predictions(model, valid_batches, val_losses,
                                           val_predictions, mode=mode)))

Few Random Images

In [ ]:
plot_predictions('random')

Random Correct Images

In [ ]:
plot_predictions('correct_random')

Random Incorrect Images

In [ ]:
plot_predictions('incorrect_random')

Most Confidently Correct

In [ ]:
plot_predictions('correct_confident')

Most Confidently Incorrect

In [ ]:
plot_predictions('incorrect_confident')

Least Confidently Correct

In [ ]:
plot_predictions('correct_doubtful')

Most Confused

In [ ]:
plot_predictions('confused')

# Analytics

In [ ]:
plot_prediction_density(model, valid_batches)

In [ ]:
print(plot_confusion_matrix(model, valid_batches))